In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install codes
!pip install hgtk
# !pip install gensim

  Created wheel for hgtk: filename=hgtk-0.1.3-py2.py3-none-any.whl size=6688 sha256=98264e6cd472c9a95e5c2f5c7682354dab57c43379db6c9ceef8f70d556b0db7
  Stored in directory: /root/.cache/pip/wheels/73/72/06/6065a57fe68264f35d7e52e37f56831eb3e9ec75656880de20
Successfully built hgtk


In [ ]:
import codecs
import hgtk
import os
import numpy as np
import pandas as pd
import csv
from gensim.models import FastText
from gensim.models.keyedvectors import KeyedVectors
import logging

EMPTY_JS_CHAR = "@"  # 사전에 약속한 특수 문자
root_dir = "../gdrive/MyDrive/AI/hackathon_2nd" 

In [ ]:
# 데이터를 입력 받아 음절 시퀀스를 저장하는 함수
# data 형태 예시 : 366	새 벽 출 조 시 <SP> 야 영 적 극 <SP> 권 장 하 ㅂ 니 다 <SP> .	B_TI I_TI O O O <SP> O O O O <SP> O O O O O O <SP> O
# file_name : 텍스트 파일 이름
def data_to_eumjeol(file_name): 
    df = pd.read_csv(f'{os.path.join(root_dir, 'data', file_name)}',
                        sep='\t',
                        quoting=csv.QUOTE_NONE,
                        header=None,
                        names=['sentence', 'ner'],
                        index_col=0)
    
    eumjeol_list = []  # dataframe의 하나의 data 마다 음절 시퀀스를 저장할 리스트
    
    for sentence in df['sentence']:
      eumjeol_list.append(sentence)  # 음절 시퀀스 저장

    return eumjeol_list

In [ ]:
# 음절 시퀀스를 입력으로 받아, 자모로 분해하는 함수(초중성 단위가 없는 문자는 특수 문자 @를 보충함) -> 모든 문자를 초성, 중성, 종성 단위로 분해
# ex : 안녕하세요 -> ㅇㅏㄴ ㄴㅕㅇ ㅎㅏ@ ㅅㅔ@ ㅇㅛ@ 
#      <SP> -> <SP>@@,  123 -> 1@@ 2@@ 3@@
def jamo_split(sentence):
    result = []
    for word in sentence.split(' '):
        decomposed_word = ""
        if word == "<SP>":
          decomposed_word += (word + "@@")
        else :     
          for char in word:
              try:
                  cho_joong_jong = hgtk.letter.decompose(char)  # 하나의 문자를 초/중/종성 단위로 분해
                  char_seq = ""
                  for cvc in cho_joong_jong:
                      if cvc == '':  
                          cvc = EMPTY_JS_CHAR  # 중성 또는 종성이 존재하지 않는 경우, 특수문자 @로 보충
                      char_seq += cvc
                  decomposed_word += char_seq
              except hgtk.exception.NotHangulException:  # 초중성 단위가 없는 문자(특수문자, 알파벳, 숫자 등)
                  decomposed_word += char
                  decomposed_word += "@@"
                  continue
        result.append(decomposed_word)
    return " ".join(result)

In [ ]:
# 텍스트 파일을 입력으로 받아, 각 data의 문장을 자모 단위로 분해하는 함수
def data_to_jamo(file_name):
    eumjeol_list = data_to_eumjeol(os.path.join(root_dir, 'data', file_name))
    data_jamo_list = []
    
    for eumjeol_seq in eumjeol_list:
       jamo_sequence = jamo_split(eumjeol_seq)
       data_jamo_list.append(jamo_sequence)

    return data_jamo_list

In [ ]:
jamo_list = data_to_jamo('ner_train.txt')
jamo_list[:5] 

['ㅅㅐ@ ㅂㅕㄱ ㅊㅜㄹ ㅈㅗ@ ㅅㅣ@ <SP>@@ ㅇㅑ@ ㅇㅕㅇ ㅈㅓㄱ ㄱㅡㄱ <SP>@@ ㄱㅝㄴ ㅈㅏㅇ ㅎㅏ@ ㅂ@@ ㄴㅣ@ ㄷㅏ@ <SP>@@ .@@',
 'ㅎㅏㄴ ㅍㅕㄴ <SP>@@ ㄷㅐ@ ㅍㅛ@ ㅌㅣㅁ ㅇㅡㄴ <SP>@@ ㅇㅣ@ ㄴㅏㄹ <SP>@@ ㄱㅓㄴ ㄱㅜㄱ ㄷㅐ@ ㅇㅘ@ <SP>@@ 3@@ ㅅㅔ@ ㅌㅡ@ ㄹㅗ@ <SP>@@ ㄴㅏ@ ㄴㅜ@ ㅇㅓ@ ㅅㅓ@ <SP>@@ ㅈㅣㄴ ㅎㅐㅇ ㄷㅚ@ ㄴ@@ <SP>@@ ㅇㅕㄴ ㅅㅡㅂ ㄱㅕㅇ ㄱㅣ@ ㅇㅔ@ ㅅㅓ@ <SP>@@ ㄱㅣㅁ ㅈㅣㄴ ㅇㅛㅇ ㄱㅘ@ <SP>@@ ㅇㅣ@ ㄷㅗㅇ ㄱㅜㄱ ,@@ <SP>@@ ㅂㅐㄱ ㅈㅣ@ ㅎㅜㄴ ㅇㅣ@ <SP>@@ ㄴㅏ@ ㄹㅏㄴ ㅎㅣ@ <SP>@@ ㄱㅗㄹ ㅇㅡㄹ <SP>@@ ㅌㅓ@ ㅌㅡ@ ㄹㅣ@ ㅇㅓ@ <SP>@@ 3@@ -@@ 0@@ <SP>@@ ㅅㅡㅇ ㄹㅣ@ ㄹㅡㄹ <SP>@@ ㄱㅓ@ ㄷㅜ@ ㅇㅓㅆ ㄷㅏ@ .@@',
 "'@@ ㅊㅓㄴ ㅈㅐ@ <SP>@@ ㄱㅗㄹ ㅈㅏㅂ ㅇㅣ@ '@@ <SP>@@ ㅂㅏㄱ ㅈㅜ@ ㅇㅕㅇ (@@ 2@@ 0@@ .@@ ㅅㅓ@ ㅇㅜㄹ )@@ ㅇㅣ@ <SP>@@ ㅍㅡ@ ㄹㅗ@ ㅊㅜㄱ ㄱㅜ@ <SP>@@ K@@ ㄹㅣ@ ㄱㅡ@ ㅇㅔ@ ㅅㅓ@ <SP>@@ 4@@ 9@@ ㅇㅣㄹ ㅁㅏㄴ ㅇㅔ@ <SP>@@ '@@ ㄱㅗㄹ ㅍㅗㄱ ㅍㅜㅇ '@@ ㅇㅡㄹ <SP>@@ ㅁㅗㄹ ㅇㅏ@ ㅊㅣ@ ㅁㅕㄴ ㅅㅓ@ <SP>@@ ㅈㅓㅇ ㄱㅠ@ ㄹㅣ@ ㄱㅡ@ <SP>@@ ㄷㅡㄱ ㅈㅓㅁ ㅇㅘㅇ ㅇㅡㄹ <SP>@@ ㅎㅑㅇ ㅎㅏ@ ㅇㅏ@ <SP>@@ ㅂㅗㄴ ㄱㅕㄱ ㅈㅓㄱ ㅇㅣ@ ㄴ@@ <SP>@@ ㄱㅗㄹ ㅅㅏ@ ㄴㅑㅇ ㅇㅔ@ <SP>@@ ㅈㅐ@ ㅅㅣ@ ㄷㅗㅇ ㅇㅡㄹ <SP>@@ ㄱㅓㄷ ㅇㅓㅆ ㄷㅏ@ .@@",
 'ㅇㅣ@ ㄱㅓㄴ ㅎㅢ@ <SP>@@ ㅈㅓㄴ <SP>@@ ㅎㅚ@ ㅈㅏㅇ ,@@ <SP>@@ "@@ ㅁㅗ@ ㄷㅡㄴ <SP>@@ ㄱㅜㄱ ㅁㅣㄴ ㅇㅣ@ <SP>@@ ㅈㅓㅇ ㅈㅣㄱ ㅎㅏ@ ㅇㅏㅆ ㅇㅡ@ ㅁㅕㄴ <SP>@@ ㅈㅗㅎ ㄱㅔㅆ ㄷㅏ@ "@@ .@@',
 'ㅁㅣㄴ ㅈㅜ@ ㄴㅗ@ ㄷㅗㅇ ㄷ

In [ ]:
hgtk.letter.decompose('요') 

('ㅇ', 'ㅛ', '')

In [ ]:
def train_fasttext_model(jamo_list):
  output_dir = os.path.join(root_dir, "fasttext_model_output")
  if not os.path.exists(output_dir):
      os.makedirs(output_dir)

  model_name = os.path.join(output_dir, 'fasttext_model')
  logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

  print("학습 중") 
  model = FastText(sentences=jamo_list, size=100, workers=4, sg=1, iter=2, word_ngrams=5, min_count=1) 
  model.save(model_name)
  print(f"학습 소요 시간 : {model.total_train_time}")  
  model.wv.save_word2vec_format(model_name + "_vis") 
  print("학습 완료")

  return model

2020-12-02 08:53:09,943 : INFO : collecting all words and their counts
2020-12-02 08:53:09,945 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2020-12-02 08:53:09,947 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


학습 중


2020-12-02 08:53:10,250 : INFO : collected 799 word types from a corpus of 2418366 raw words and 7319 sentences
2020-12-02 08:53:10,250 : INFO : Loading a fresh vocabulary
2020-12-02 08:53:10,254 : INFO : effective_min_count=1 retains 799 unique words (100% of original 799, drops 0)
2020-12-02 08:53:10,255 : INFO : effective_min_count=1 leaves 2418366 word corpus (100% of original 2418366, drops 0)
2020-12-02 08:53:10,262 : INFO : deleting the raw counts dictionary of 799 items
2020-12-02 08:53:10,263 : INFO : sample=0.001 downsamples 31 most-common words
2020-12-02 08:53:10,264 : INFO : downsampling leaves estimated 493284 word corpus (20.4% of prior 2418366)
2020-12-02 08:53:10,271 : INFO : estimated required memory for 799 words, 799 buckets and 100 dimensions: 1403044 bytes
2020-12-02 08:53:10,272 : INFO : resetting layer weights
2020-12-02 08:53:10,513 : INFO : Total number of ngrams is 799
2020-12-02 08:53:10,526 : INFO : training model with 4 workers on 799 vocabulary and 100 fe

학습 소요 시간 : 4.82961970300039
학습 완료


In [ ]:
model.wv.vectors.shape

(799, 100)

In [ ]:
model.wv.index2word

['@',
 ' ',
 'ㅇ',
 'S',
 'P',
 '>',
 '<',
 'ㅏ',
 'ㄴ',
 'ㄱ',
 'ㄹ',
 'ㅣ',
 'ㅡ',
 'ㅓ',
 'ㅅ',
 'ㅗ',
 'ㄷ',
 'ㅈ',
 'ㅎ',
 'ㅁ',
 'ㅜ',
 'ㅂ',
 'ㅔ',
 'ㅕ',
 'ㅐ',
 'ㅊ',
 'ㅆ',
 '.',
 'ㅌ',
 'ㅢ',
 'ㅘ',
 'ㅍ',
 'ㅚ',
 ',',
 'ㅛ',
 'ㅝ',
 '1',
 'ㅋ',
 '0',
 '2',
 'ㅠ',
 'ㅟ',
 'ㅑ',
 'ㄲ',
 'ㄸ',
 'ㅖ',
 '(',
 ')',
 '3',
 "'",
 '5',
 '4',
 '"',
 '6',
 'ㄶ',
 '7',
 '8',
 '9',
 'ㅄ',
 'ㅉ',
 '-',
 'ㅃ',
 'ㄺ',
 '%',
 'A',
 'C',
 'e',
 '·',
 'o',
 'B',
 'K',
 'a',
 'T',
 'm',
 'i',
 'L',
 't',
 'r',
 'n',
 ':',
 'G',
 'M',
 'I',
 'w',
 '’',
 'c',
 '‘',
 'F',
 '~',
 'E',
 's',
 'O',
 'D',
 'ㄻ',
 'ㅞ',
 'V',
 '/',
 'l',
 '[',
 ']',
 'k',
 'N',
 'ㅀ',
 '“',
 '”',
 'ㅙ',
 '?',
 'u',
 'R',
 'd',
 'ㅒ',
 'g',
 '`',
 'y',
 'p',
 'ㄵ',
 'ㄼ',
 'U',
 '!',
 'v',
 'h',
 'b',
 'ㆍ',
 'H',
 'X',
 '民',
 'J',
 '…',
 '&',
 'Y',
 '∼',
 'W',
 '神',
 'f',
 '美',
 '▲',
 '國',
 '的',
 '性',
 '黨',
 '正',
 'x',
 '+',
 '佛',
 '理',
 '人',
 '大',
 'z',
 'ｍ',
 '．',
 ';',
 '㎝',
 '道',
 'ㄳ',
 '學',
 '化',
 '{',
 '}',
 '㈜',
 '山',
 '日',
 '*',
 '生',
 '政',
 '敎',
 '㎏',
 '子'

In [ ]:
len(model.wv.index2word)

799

In [ ]:
model.wv['!']

array([-0.07275349, -0.00551543,  0.09301741,  0.0548504 , -0.00250614,
        0.07871351,  0.08319694,  0.09489648,  0.03938426, -0.06807365,
       -0.02281412,  0.00658904,  0.01360555, -0.03238269, -0.0568224 ,
        0.04221815, -0.00230448, -0.05668335, -0.03813361, -0.0122227 ,
       -0.05553164, -0.06983908, -0.05585964,  0.00502901, -0.05087065,
       -0.08159201,  0.06588542, -0.01534844, -0.06426921, -0.02344786,
        0.04466565,  0.00016827,  0.00420104, -0.09910724,  0.04991522,
       -0.09648036,  0.05195775, -0.02455254, -0.04798845, -0.02402876,
       -0.02645341, -0.03472355,  0.00495086,  0.06958833, -0.05316784,
        0.02882657, -0.09003635,  0.01721793,  0.03371263, -0.09486476,
       -0.1423808 ,  0.10969562, -0.09834839, -0.0133746 ,  0.00762217,
       -0.10244711, -0.02346428,  0.03578093,  0.13695663, -0.02403381,
        0.02380204, -0.0403838 ,  0.048012  ,  0.03747752, -0.04472947,
       -0.04230446,  0.0366461 , -0.01240102,  0.0051276 , -0.10

In [ ]:
print(type(model.wv['!']))

<class 'numpy.ndarray'>


In [ ]:
# 해야할 일
# 1. fasttext word vocab txt 파일 생성
# 2. 어절 마다 자모 단위로 분해한, train 및 validation 파일 만들기
# 3. load_data 함수 기능 추가 (자모 단위로 분해한 문자에 대해 임베딩 작업을 수행) 

In [ ]:
# FastText 모델 불러오기
load_model = FastText.load(os.path.join(root_dir, "fasttext_model_output", 'fasttext_model'))

In [ ]:
# 1. fasttext word vocab txt 파일 생성
def create_fasttext_vocab(model):
    with open(os.path.join(root_dir, "data", 'vocab_for_fasttext.txt'), 'w', encoding='utf8') as f:
        for char in model.wv.index2word:
            f.write('{}\n'.format(char))

create_fasttext_vocab(load_model) # vocab_size : 799

In [ ]:
# 2. 어절 마다 자모 단위로 분해한 train(validation) 또는 test 파일 만들기
def jamo_seq_from_data(model, mode, source_file_name, target_file_name):       
    with open(os.path.join(root_dir, target_file_name), 'w', encoding='utf8') as f1:
        if mode == 'train':
            df = pd.read_csv(f'{os.path.join(root_dir, "data", source_file_name)}',
                        sep='\t',
                        quoting=csv.QUOTE_NONE,
                        header=None,
                        names=['sentence', 'ner'],
                        index_col=0, 
                        encoding='utf8')            

        elif mode == 'test':
            df = pd.read_csv(f'{os.path.join(root_dir, "data", source_file_name)}',
                        sep='\t',
                        quoting=csv.QUOTE_NONE,
                        header=None,
                        names=['sentence'],
                        index_col=0, 
                        encoding='utf8')
            
        id_list = df.index
            
        for id in id_list:
            sentence = df.loc[id]['sentence']
            # sentence : 새 벽 출 조 시 <SP> 야 영 적 극 <SP> 권 장 하 ㅂ 니 다 <SP> .
            # jamo_sequence: ㅅㅐ@ ㅂㅕㄱ ㅊㅜㄹ ㅈㅗ@ ㅅㅣ@ <SP>@@ ㅇㅑ@ ㅇㅕㅇ ㅈㅓㄱ ㄱㅡㄱ <SP>@@ ㄱㅝㄴ ㅈㅏㅇ ㅎㅏ@ ㅂ@@ ㄴㅣ@ ㄷㅏ@ <SP>@@ .@@'
            jamo_sequence = jamo_split(sentence)  
            df.loc[id]['sentence'] = jamo_sequence

        df.to_csv(f'{os.path.join(root_dir, "data", target_file_name)}', 
                  sep='\t',
                  header=None, 
                  quoting=csv.QUOTE_NONE, 
                  escapechar='\\')

In [ ]:
# train.txt 파일 (음절 시퀀스 -> 자모 시퀀스, 나머지는 그대로) 재구성하여 새로 저장
jamo_seq_from_data(model=load_model, mode='train', source_file_name='ner_train.txt', target_file_name='ner_train_jamo.txt')

In [ ]:
# dev.txt 파일 (음절 시퀀스 -> 자모 시퀀스, 나머지는 그대로) 재구성하여 새로 저장
jamo_seq_from_data(model=load_model, mode='train', source_file_name='ner_dev.txt', target_file_name='ner_dev_jamo.txt')